# なぜ継承より合成なのか？
名前空間をごっちゃにするから。  

継承とはコードを共通化すること。コードを共通化することの危険性を動画にして、分かりやすく説明している。以下の動画の共通モジュールを親クラスに読み換えると雰囲気が掴みやすい。
(動画：[共通化の罠](https://twitter.com/MinoDriven/status/1127539251761909760)）  

いつ継承でいつ合成を使うのか？（とてもざっくり）

| 内容 | 継承 | 合成 |
| --- | --- | --- |
| 用途 | 小規模 | 大規模 |

規模が小さい、少人数の時はシステムが持つクラスの全体像が見渡せるので、継承をして名前空間をごっちゃにしても問題は大きくないかと思う。規模が大きい、大人数の時は、システムが持つクラスの全体像を見渡すことは、難しいので継承より合成を使い名前空間をきっちり分けた方が、責任の分解点が明瞭になるかと思う。

継承と合成のメリット、デメリット

| 内容 | 継承 | 合成 |
| --- | --- | --- |
| 実装 | 容易 | 面倒 |
| 名前空間 | ごっちゃ | 分けられる |

継承は、簡単に実装を共有できる。合成は処理を委譲するコードを書かなければならず煩雑。継承は簡単に実装できる反面、名前空間を継承した先のクラスと共有してしまい、責任の分解点が曖昧になる。委譲は合成の実装が面倒だが、名前空間をきっちり分けることができる。

## 1. 合成と委譲
合成とは属性に持たせたい機能を持つオブジェクトを代入すること。委譲とは合成で代入されたオブジェクトをメソッドから呼び出すこと。

In [2]:
# 継承 inheritance
class Team(list):
    pass

team = Team()
team.append('Tora')
team 

['Tora']

In [3]:
# 合成 composition
class Team:
    def __init__(self):
        self._list = []
    
    # 委譲
    def add(self, general):
        self._list.append(general)

team = Team()
team.add('Tora')

> コンポジション(Composition)は、日本語で「混合物」を意味する単語である。あるクラスの機能を持つクラスのことを指す。特定のクラスの機能を、自分が作るクラスにも持たせたい場合に、継承を使わずフィールドとしてそのクラスを持ち、クラスのメソッドを呼び出すメソッドを持たせること。そうすることで、クラスに他のクラスの機能を組み込むことができる。

[継承とコンポジションをどう使い分けるか - いぬごやねっと](http://4geek.net/difference-between-inheritance-and-composition/)


## 1.1. 合成 composition
持たせたい機能を属性に代入することを合成と言う。  
さて先頭にアンダーバーのついた不思議な属性名`_list`を指定していた。これはなんだろう？
```
def __init__(self):
    self._list = [] # <- ここ
```
Pythonのコーディング規約PEP8では、クラスのメソッドの外から参照されたくない属性は、先頭にアンダーバーをつけるように指定されている。
```
# インスタンス化した属性を直接見られたくない時に使う
team = Team()
team._list.append('とら')
```

## 1.2. 委譲 deligation
合成でさえ面倒なのに、なぜわざわざaddという関数を使って呼び出しているのだろうか？ 直接、属性を操作してしまえば良いのではないか？

```
# 合成 composition
class Team:
    def __init__(self):
        self._list = []

team = Team()
team._list.append('とら')  # <- これで良くない？
```
おそらく結合度が上がるのを避けるために行われるのだと思う。結合度とは、ごくざっくり言えば、`.`で属性参照をするほど結合度が高いコードと見なされる。なぜ、こう手間をかけてまで結合度を上げるのを避けたいのでしょうか？

addという関数を用意することで、もしappendに変更があった場合にはaddだけを変えれば良くなる。直接self._list.appendを呼び出してしまうと、呼び出している先、全てのコードで変更が必要になる。